# Word Embedding

#### Embedding Vector 시각화 webi
https://ronxin.github.io/wevi/

- **Word Embedding**은 단어를 고정된 차원의 벡터로 변환하는 기술로, 단어 간의 의미적 유사성을 반영하도록 학습된 벡터를 말한다.
- 이 기술은 자연어 처리에서 문장을 처리하고 이해하는 데 활용된다.
- 숫자로 표현된 단어 목록을 통해 감정을 추출하는 것도 가능하다.
- 연관성 있는 단어들을 군집화하여 다차원 공간에 벡터로 나타낼 수 있으며, 이는 단어나 문장을 벡터 공간에 매핑하는 과정이다.

**Embedding Matrix 예시**

*아래 표의 벡터 값들은 모두 기계 학습을 통해 학습된 결과이다.*  

| Dimension | Man (5391) | Woman (9853) | King (4914) | Queen (7157) | Apple (456) | Orange (6257) |
|-----------|------------|--------------|-------------|--------------|-------------|---------------|
| 성별      | -1         | 1            | -0.95       | 0.97         | 0.00        | 0.01          |
| 귀족      | 0.01       | 0.02         | 0.93        | 0.95         | -0.01       | 0.00          |
| 나이      | 0.03       | 0.02         | 0.7         | 0.69         | 0.03        | -0.02         |
| 음식      | 0.04       | 0.01         | 0.02        | 0.01         | 0.95        | 0.97          |

<br>

*아래는 전치된 표이다.*

| Word          | 성별   | 귀족   | 나이   | 음식   |
|---------------|--------|--------|--------|--------|
| Man (5391)    | -1.00  | 0.01   | 0.03   | 0.04   |
| Woman (9853)  | 1.00   | 0.02   | 0.02   | 0.01   |
| King (4914)   | -0.95  | 0.93   | 0.70   | 0.02   |
| Queen (7157)  | 0.97   | 0.95   | 0.69   | 0.01   |
| Apple (456)   | 0.00   | -0.01  | 0.03   | 0.95   |
| Orange (6257) | 0.01   | 0.00   | -0.02  | 0.97   |

In [3]:
!pip install gensim

##### 영어 Word Embedding

- 데이터 취득 및 전처리

In [5]:
!pip install gdown

In [6]:
import gdown

url = 'https://drive.google.com/uc?id=1TF1yAHF3qRINbXWFOajFjUCxUF64QZMX'
output = 'ted_en.xml'

gdown.download(url, output)

FileURLRetrievalError: Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1TF1yAHF3qRINbXWFOajFjUCxUF64QZMX

but Gdown can't. Please check connections and permissions.

In [ ]:
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

In [ ]:
f = open('ted_en.xml', 'r', encoding='UTF-8')
xml = etree.parse(f)

contents = xml.xpath('//content/text()')    # content 태그 하위 텍스트
# contents[:3]

corpus = '\n'.join(contents)
print(len(contents))
# print(contents[3])

# 정규표현식을 이용해 (Laughter), (Applause) 등 제거
corpus = re.sub(r'\([^)]*\)', '', corpus)
print(corpus)

In [ ]:
# 데이터 전처리 (토큰화, 대소문자 정규화, 불용어 처리)
sentences = sent_tokenize(corpus)

preprocessed_sentences = []
en_stopwords = stopwords.words('english')

for sentence in sentences:
    sentence = sentence.lower()
    sentence = re.sub(r'[^a-z0-9]', ' ', sentence)
    tokens = word_tokenize(sentence)
    tokens = [token for token in tokens if token not in en_stopwords]
    preprocessed_sentences.append(tokens)

preprocessed_sentences[:5]

- Embedding 모델 학습

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(
    sentences=preprocessed_sentences,   # corpus
    vector_size=100,                    # 임베딩 벡터 차원
    sg=0,                               # 학습 알고리즘 선택 (0:CBOW, 1:Skip-gram)
    window=5,                           # 주변 단어 수 (앞뒤로 n개 고려)
    min_count=5                         # 최소 빈도 (빈도 n개 미만은 제거)
)

model.wv.vectors.shape

In [ ]:
import pandas as pd

pd.DataFrame(model.wv.vectors, index=model.wv.index_to_key).head(10)

In [ ]:
# 학습된 단어 임베딩 저장
model.wv.save_word2vec_format('ted_en_w2v')

In [ ]:
# 임베딩 모델 로드
from gensim.models import KeyedVectors

load_model = KeyedVectors.load_word2vec_format('ted_en_w2v')

- 유사도 계산

In [ ]:
# model: Word2Vec
model.wv.most_similar('man')
# model.wv.most_similar('asneklej') # 임베딩 벡터에 없는 단어로 조회 시 KeyError 발생

In [ ]:
# load_model: KeyedVectors == Word2Vec.wv
load_model.most_similar('man')

In [ ]:
model.wv.similarity('man', 'husband')

In [ ]:
model.wv['man']

- 임베딩 시각화

In [ ]:
!python -m gensim.scripts.word2vec2tensor --input ted_en_w2v --output ted_en_w2v

### 한국어 Word Embedding

- NSMC (Naver Sentiment Movie Corpus)

In [ ]:
# 데이터 다운로드
import urllib.request

urllib.request.urlretrieve(
    'https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt',
    filename='naver_movie_ratings.txt'
)

In [ ]:
# 데이터 프레임 생성
ratings_df = pd.read_csv('naver_movie_ratings.txt', sep='\t')

In [ ]:
ratings_df.isnull().sum()

ratings_df = ratings_df.dropna(how='any')

In [ ]:
ratings_df['document'][200:300]

In [ ]:
ratings_df['document'] = ratings_df['document'].replace(r'[^0-9가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', regex=True)

In [ ]:
from konlpy.tag import Okt
from tqdm import tqdm

okt = Okt()
ko_stopwords = ['은', '는', '이', '가', '을', '를', '과', '와', '들', '도', '부터', '까지', '에', '나', '너', '그', '걔', '얘']

preprocessed_data = []

for sentence in tqdm(ratings_df['document']):
    tokens = okt.morphs(sentence, stem=True)
    tokens = [token for token in tokens if token not in ko_stopwords]
    preprocessed_data.append(tokens)

In [ ]:
model = Word2Vec(
    sentences=preprocessed_data,
    vector_size=100,
    window=5, 
    min_count=5,
    sg=0
)

model.wv.vectors.shape

In [ ]:
model.wv.most_similar('극장')

In [ ]:
model.wv.similarity('김혜수', '유해진')

In [ ]:
# 모델 저장
model.wv.save_word2vec_format('naver_movie_ratings_w2v')

In [ ]:
!python -m gensim.scripts.word2vec2tensor --input naver_movie_ratings_w2v --output naver_movie_ratings_w2v

- 사전 훈련된 임베딩

In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=1aL_xpWW-CjfCrLWeflIaipITOZ6zHI5c'
output = 'GoogleNews_vecs.bin.gz'

gdown.download(url, output)

In [ ]:
google_news_wv = KeyedVectors.load_word2vec_format('GoogleNews_vecs.bin.gz', binary=True)
google_news_wv.vectors.shape

In [ ]:
google_news_wv.similarity('man', 'husband')

In [ ]:
google_news_wv.most_similar('man', topn=5)

In [ ]:
# 두 리스트 간의 평균 유사도 계산
google_news_wv.n_similarity(['king', 'queen'], ['man', 'woman'])

In [ ]:
google_news_wv.similar_by_word('man', topn=5)

In [ ]:
google_news_wv.has_index_for('man')